# ST-HDBSCAN: Spatiotemporal Hierarchical DBSCAN for Trajectory Data

## Abstract

The study of human mobility has advanced greatly in recent years due to the availability of
commercial large-scale GPS trajectory datasets [3]. However, the validity of findings that use
these datasets depends heavily on the robustness of their pre-processing methods. An important
step in the processing of mobility data is the detection of stops within GPS trajectories, for
which many clustering algorithms have been proposed [4, 8, 6, 1]. Yet, the high sparsity of
commercial GPS data can affect the performance of these stop-detection algorithms.
In the case of DBSCAN, while it initially identifies dense regions, it can often over-cluster
or under-cluster due to noise and weakly connected points given the chosen ε. ST-DBSCAN [4]
uses two distance thresholds, Eps1 for spatial and Eps2 for non-spatial values. The algorithm
compares the average non-spatial value, such as temperature, of a cluster with a new com-
ing value, to prevent merging adjacent clusters. Nevertheless, datasets that include this kind
of information are not comparable to realistic GPS-based trajectories. A promising algorithm
is T-DBSCAN [6], which searches forward in time for a continuous density-based neighbor-
hood of core points. Points spatially close, within Eps, and within a roaming threshold, CEps, are included in a cluster. Additionally, we used a time-augmented DBSCAN algorithm, TA-DBSCAN, which recursively processes the clusters obtained from DBSCAN to address the issue of initial clusters overlapping in time. However, methods [9] that validate stop-detection algorithms based on synthetic data show that these can omit, merge, or split stops based on the selection of epsilon and sparsity of the data.

If we define parameters that may be considered fine (low ε), it might completely miss a stop at a larger location. In contrast, coarse parameters (large ε) may struggle to differentiate stops within small neighboring locations [3]. Since different venues vary in stop durations and areas, this could influence the parameter choices [9]. To address this parameter selection limitation, we propose a spatiotemporal variation of Hierarchical DBSCAN [5], ST-HDBSCAN. Unlike DBSCAN, which relies on one threshold of density to cluster points, our variation constructs separate structures for space and time distances that preserve density-based connections in these two dimensions. This approach ensures that when pruning the hierarchical tree structure needed for cluster formation, we account for varying spatiotemporal densities. As a result, clusters emerge naturally without requiring specific time and space thresholds, working effectively across different data sparsity levels.

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import pygeohash as gh
import geopandas as gpd
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from pyproj import Transformer
import heapq
from collections import defaultdict

In [2]:
import nomad.io.base as loader
import nomad.constants as constants
import nomad.stop_detection.ta_dbscan as DBSCAN
import nomad.stop_detection.lachesis as Lachesis
import nomad.filters as filters
import nomad.city_gen as cg

In [3]:
def find_bursts(times, time_thresh, burst_col = False):
    # Pairwise time differences
    time_diffs = np.abs(times[:, np.newaxis] - times)
    time_diffs = time_diffs.astype(int)
    
    # Filter by time threshold
    within_time_thresh = np.triu(time_diffs <= (time_thresh * 60), k=1)
    i_idx, j_idx = np.where(within_time_thresh)
    
    # Return a list of (timestamp1, timestamp2) tuples
    time_pairs = [(times[i], times[j]) for i, j in zip(i_idx, j_idx)]
    
    return time_pairs

In [4]:
def compute_core_distance_with_time(traj, bursts, min_samples = 2):
    """
    Calculate the core distance for each ping in traj.

    Parameters
    ----------
    traj : dataframe
    
    min_samples : int
        used to calculate the core distance of a point p, where core distance of a point p 
        is defined as the distance from p to its min_samples-th smallest nearest neighbor
        (including itself).

    Returns
    -------
    core_distances : dictionary of timestamps
        {timestamp_1: core_distance_1, ..., timestamp_n: core_distance_n}
    """
    # it gives local density estimate: small core distance → high local density.
    coords = traj[['x', 'y']].to_numpy()
    timestamps = traj['timestamp'].to_numpy()
    n = len(coords)
    timestamps_dict = {ts: idx for idx, ts in enumerate(timestamps)}

    # Build neighbor map from bursts
    neighbor_map = {ts: set() for ts in timestamps}
    
    for t1, t2 in bursts:
        neighbor_map[t1].add(t2)
        neighbor_map[t2].add(t1)

    core_distances = {}
    
    for i in range(n):
        ts_i = timestamps[i]
        allowed_neighbors = neighbor_map[ts_i]
        dists = np.full(n, np.inf)
        dists[i] = 0  # distance to itself
        
        for ts_j in allowed_neighbors:
            j = timestamps_dict.get(ts_j)
            if j is not None:
                dists[j] = np.sqrt(np.sum((coords[i] - coords[j]) ** 2))
        
        sorted_dists = np.sort(dists)
        core_distances[ts_i] = sorted_dists[min_samples - 1]

    return core_distances

In [5]:
def compute_core_distance(traj, min_samples = 2):
    """
    Calculate the core distance for each ping in traj.

    Parameters
    ----------
    traj : dataframe
    
    min_samples : int
        used to calculate the core distance of a point p, where core distance of a point p 
        is defined as the distance from p to its min_samples-th smallest nearest neighbor
        (including itself).

    Returns
    -------
    core_distances : dictionary of timestamps
        {timestamp_1: core_distance_1, ..., timestamp_n: core_distance_n}
    """
    # it gives local density estimate: small core distance → high local density.
    coords = traj[['x', 'y']].to_numpy()
    timestamps = traj['timestamp'].to_numpy()
    n = len(coords)
    core_distances = {}

    # TC: 0(n^2)
    for i in range(n):
        # pairwise euclidean distances
        dists = np.sqrt(np.sum((coords - coords[i]) ** 2, axis=1))
        # sort distances and get min_samples-th smallest
        sorted_dists = np.sort(dists)
        core_distance = sorted_dists[min_samples - 1]
        core_distances[timestamps[i]] = core_distance

    return core_distances

def compute_mrd(traj, core_distances):
    """
    Mutual Reachability Distance (MRD) of point p and point q is the maximum of: 
    cordistance of p, core distance of q, and the distance between p and q.

    Parameters
    ----------
    traj : dataframe
    
    core_distances : dict
        Keys are (timestamp1, timestamp2), values are mutual reachability distances.

    Returns
    -------
    mrd : dictionary of timestamp pairs
        {(timestamp1, timestamp2): mrd_value}.
    """
    # edges between dense regions are favored, and sparse regions have larger edge weights
    # MRD will inflate distances so that sparse areas are less likely to form clusters
    # Even if two points are physically close, we shouldn’t treat them as equally “reachable”
    # because they live in very different local densities
    coords = traj[['x', 'y']].to_numpy()
    timestamps = traj['timestamp'].to_numpy()
    n = len(coords)
    mrd = {}

    # TC: 0(n^2)
    for i in range(n):
        for j in range(i + 1, n):
            # euclidean distance between point i and j
            dist = np.sqrt(np.sum((coords[j] - coords[i]) ** 2))
            core_i = core_distances[timestamps[i]]
            core_j = core_distances[timestamps[j]]

            mrd[(timestamps[i], timestamps[j])] = max(core_i, core_j, dist)

    return mrd

def mst(mrd):
    """
    Compute the MST using Prim's algorithm from mutual reachability distances.

    Parameters
    ----------
    mrd : dict
        Keys are (timestamp1, timestamp2), values are mutual reachability distances.

    Returns
    -------
    mst_edges : list of tuples
        (timestamp1, timestamp2, mrd_value) for the MST.
    """
    graph = defaultdict(list)
    
    for (u, v), weight in mrd.items():
        graph[u].append((weight, v))
        graph[v].append((weight, u))

    visited = set()
    mst_edges = []
    start_node = next(iter(graph))
    min_heap = [(0, start_node, start_node)]  # (weight, from, to)

    while min_heap and len(visited) < len(graph):
        weight, frm, to = heapq.heappop(min_heap)
        
        if to in visited:
            continue
        
        visited.add(to)
        
        if frm != to:
            mst_edges.append((frm, to, weight))
        
        for next_weight, neighbor in graph[to]:
            if neighbor not in visited:
                heapq.heappush(min_heap, (next_weight, to, neighbor))

    return mst_edges

def mst_ext(mst_edges, core_distances):
    """
    Add self-edges to MST with weights equal to each point's core distance.
    """
    self_edges = [(ts, ts, core_distances[ts]) for ts in core_distances]
    return mst_edges + self_edges


def hdbscan(mst_ext, min_cluster_size):
    hierarchy = []
    
    # 4.1 For the root of the tree assign all objects the same label (single “cluster”).
    all_pings = set()
    
    for u, v, _ in mst_ext:
        all_pings.add(u)
        all_pings.add(v)
        
    label_map = {ts: 0 for ts in all_pings} # {'t1':0, 't2':0, 't3':0}
    active_clusters = {0: set(all_pings)} # e.g. { 0: {'t1', 't2', 't3'} }
    
    # sort edges in decraesing order of weight
    mst_ext_sorted = sorted(mst_ext, key=lambda x: -x[2]) 
    
    # group edges by weight
    dendrogram_scales = defaultdict(list)
    for u, v, w in mst_ext_sorted:
        dendrogram_scales[w].append((u, v))

    current_label_id = max(label_map.values()) + 1

    # 4.2.1 Before each removal, set the dendrogram scale value of the current hierarchical level as the weight of the edge(s) to be removed.
    for scale, edges in dendrogram_scales.items():
        affected_clusters = set()
        edges_to_remove = []
        
        for u, v in edges:
            if label_map.get(u) != label_map.get(v):
                continue
            cluster_id = label_map[u]
            affected_clusters.add(cluster_id)
            edges_to_remove.append((u, v))

        # 4.2.2: For each affected cluster, reassign components
        for cluster_id in affected_clusters:
            if cluster_id == -1 or cluster_id not in active_clusters:
                continue  # skip noise or already removed clusters
            
            members = active_clusters[cluster_id]
    
            # build connectivity graph (excluding removed edges)
            G = build_connectivity_graph(members, mst_ext, edges_to_remove)
            components = connected_components(G)
            non_spurious = [c for c in components if len(c) >= min_cluster_size]

            # cluster has disappeared
            if not non_spurious:
                for ts in members:
                    label_map[ts] = -1  # noise
                del active_clusters[cluster_id]
            # cluster has just shrunk
            elif len(non_spurious) == 1:
                remaining = non_spurious[0]
                for ts in members:
                    label_map[ts] = cluster_id if ts in remaining else -1
                active_clusters[cluster_id] = remaining
            # true cluster split: multiple valid subclusters
            elif len(non_spurious) > 1:
                new_ids = []
                del active_clusters[cluster_id]
                for comp in non_spurious:
                    for ts in comp:
                        label_map[ts] = current_label_id
                    active_clusters[current_label_id] = set(comp)
                    new_ids.append(current_label_id)
                    current_label_id += 1
                
                hierarchy.append((scale, cluster_id, new_ids))

    return {"label_map": label_map, "hierarchy": hierarchy}

def build_connectivity_graph(nodes, edges, removed_edges):
    # nodes: set of timestamps {t1,t2,t3} 
    # edges: list of (u, v, w) tuples
    # removed_edges: list of (u,v) tuples
    graph = defaultdict(set)
    removed_set = set(frozenset(e) for e in removed_edges)
    for u, v, _ in edges:
        if frozenset((u, v)) in removed_set:
            continue
        if u in nodes and v in nodes and u != v:
            graph[u].add(v)
            graph[v].add(u)
    return graph

def connected_components(graph):
    seen = set()
    components = []

    for node in graph:
        if node in seen:
            continue
        stack = [node]
        comp = set()
        while stack:
            n = stack.pop()
            if n not in seen:
                seen.add(n)
                comp.add(n)
                stack.extend(graph[n] - seen)
        components.append(comp)

    return components

In [6]:
traj_cols = {'user_id':'uid',
             'datetime':'local_datetime',
             'latitude':'latitude',
             'longitude':'longitude'}

data = loader.from_file("../../nomad/data/gc_sample.csv")

In [ ]:
# We create a time offset column with different UTC offsets (in seconds)
data['tz_offset'] = 0
data.loc[data.index[:5000],'tz_offset'] = -7200
data.loc[data.index[-5000:], 'tz_offset'] = 3600

# create datetime column as a string
data['local_datetime'] = loader._unix_offset_to_str(data.timestamp, data.tz_offset)
data['local_datetime'] = pd.to_datetime(data['local_datetime'], utc=True)

# create x, y columns in web mercator
gdf = gpd.GeoSeries(gpd.points_from_xy(data.longitude, data.latitude),
                        crs="EPSG:4326")
projected = gdf.to_crs("EPSG:3857")
data['x'] = projected.x
data['y'] = projected.y

In [ ]:
data

In [ ]:
user_sample = data.loc[data.uid == "angry_spence"]
user_sample = user_sample[['timestamp', 'x', 'y']]

In [ ]:
user_sample

In [ ]:
core_distances = compute_core_distance(user_sample, 4)
mrd = compute_mrd(user_sample, core_distances)
mst_edges = mst(mrd)
mstext_edges = mst_ext(mst_edges, core_distances)
hdbscan(mstext_edges, 5)